### Importing Source Files

In [0]:
%run "../Includes/Configuration"

In [0]:
%run "../Includes/Common_Functions"

In [0]:
%run "../Includes/sas alternative"

# Drivers File

In [0]:
df_driver = spark.read.parquet(f"{Processed_path}/Drivers")

In [0]:
# Renaming The Columns
df_driver = df_driver.withColumnRenamed('number', 'driver_number')\
       .withColumnRenamed('name', 'driver_name')\
       .withColumnRenamed('nationality', 'driver_nationality')
df_driver.display()

# Constructers

In [0]:
df_Constructers = spark.read.parquet(f"{Processed_path}/New_Constructer")

In [0]:
# Renming The Columns
df_Constructers = df_Constructers.withColumnRenamed('constructorId','constructor_Id')\
                                 .withColumnRenamed('constructorRef', 'constructor_Ref')\
                                 .withColumnRenamed('name', 'team')
df_Constructers.display()

# Circuits

In [0]:
df_Circuits = spark.read.parquet(f"{Processed_path}/circuits") # Reading DataFrame

In [0]:
df_Circuits = df_Circuits.withColumnRenamed('location','circuits_location') # Renaming The Columns
df_Circuits.display()

# Races

In [0]:
df_Races = spark.read.parquet(f"{Processed_path}/races")

In [0]:
# Renaming The Columns
df_Races = df_Races.withColumnRenamed("name", "race_name")\
                   .withColumnRenamed('race_timestamp', 'race_date')
df_Races.display()

# Results

In [0]:
df_Result = spark.read.parquet(f"{Processed_path}/Results")                  

In [0]:
df_Result = df_Result.withColumnRenamed("time", "race_time")# Renaming the columns
df_Result.display()

## Joining The Race, Circuits Tables and getting Columns we need

In [0]:
df_Circuits_Join_Race = df_Circuits.join(df_Races, on= "circuit_id", how='inner')
df_Circuits_Join_Race.display()

In [0]:
# race id from race table
# race year from race table
# race_name from race table
# race_date from race table
# circuit location from circuits
df_Circuits_Join_Race_Specfic_Columns = df_Circuits_Join_Race.select('race_Id', 'race_year', 'race_name', 'race_date', 'circuits_location')
df_Circuits_Join_Race_Specfic_Columns.display()


### Joining Results Table with Race and Circuit

In [0]:
df_Result_Race_Circuit = df_Circuits_Join_Race_Specfic_Columns\
                         .join(df_Result, on='race_Id', how='inner')

df_Result_Race_Circuit.display()

## Circuits_Race_Driver join

In [0]:
df_Circuits_Race_Driver = df_Result_Race_Circuit.join(df_driver, on='driver_id',how='inner')
df_Circuits_Race_Driver.display()

In [0]:
df_Circuits_Race_Driver = df_Circuits_Race_Driver.withColumnRenamed('constructorId','constructor_Id')

In [0]:
df_Circuits_Race_Driver_Constructer = df_Circuits_Race_Driver.join(df_Constructers, on='constructor_Id', how='inner')

In [0]:
df_Circuits_Race_Driver_Constructer.display()

In [0]:
final_df = df_Circuits_Race_Driver_Constructer.select('race_year', 'race_name', 'race_date', 'circuits_location', 'driver_name', 'driver_number', 'driver_nationality','team', 'grid', 'fastest_Lap', 'race_time', 'points', 'position')

In [0]:
final_df.display()

In [0]:
from pyspark.sql.functions import current_timestamp
final_df = final_df.withColumn("Creation_Date",current_timestamp())

In [0]:
from pyspark.sql.functions import col, when

In [0]:
from pyspark.sql.functions import col

final_df_2 = final_df.select(col('race_name'), col('race_year'), col('points')) \
    .filter((col('race_name') == 'Abu Dhabi Grand Prix') & (col('race_year') == 2020)) \
    .orderBy(col('points').desc())

display(final_df_2)

In [0]:
final_df.write.mode('overwrite').parquet(f"{Presentation_path}/Race_Results_R")